In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from skl2onnx import to_onnx
from onnxruntime import InferenceSession
%matplotlib inline

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# Load the data
data_path = "filled_data.csv"
data = pd.read_csv(data_path)

# Extract the relevant columns
datetime_column = pd.to_datetime(data['datetime'])
# Get all feature columns except 'datetime'
feature_columns = data.columns[1:]  # Exclude the 'datetime' column
from challenge_utils import build_training_data

# Data Exploration

In [ ]:
print(data.shape)
data.head()

In [ ]:
cor_matrix = data.corr()
# cor_matrix
col = "kw_total_zone2"
sort_col_matrix = cor_matrix.sort_values(col, ascending=False)[col]
selected_feature = [x for x in sort_col_matrix.index if abs(sort_col_matrix[x]) > 0.2 and x != col]
# print(sort_col_matrix)
print(sort_col_matrix[selected_feature])

In [ ]:
day_start = 100
kw_zone2 = data["kw_total_zone2"][24*day_start:24*(day_start + 7)]
import matplotlib.pyplot as plt 
plt.plot(range(len(kw_zone2)), kw_zone2, label="kw_zone2_hourly")
# plt.figure(figsize=(20, 10))

In [ ]:
feature_zone_2 = [x for x in selected_feature if "zone1" not in x]
print(feature_zone_2)
features = ["plugs_zone2", "kw_ventilation_zone2", "kw_water_heater_zone2", "kw_lights_zone2", "kw_heaters_toilets_zone2", "kw_heaters_corridor_zone2", "kw_heatingcoolingtotal_zone2", "kw_total_zone2"]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 6), dpi=200)

time = data['datetime'].astype(np.datetime64).values.astype('datetime64[s]')

# slicing out a few months of data
start = 500
some = slice(start, start + 1000)

for col in data.columns[1::]:

    # only plotting a few variables
    if col in features:
        ax.plot(time[some], data[col].values[some], label=col)

ax.legend()

Why are there some negative features? How can the consumption be negative? \\
Why is there a part of zeros for kw_total_zone2 between 22-02 to 01-03-2022?
- Did they shut down the whole building?
- The measure doesn't working?
- They took out the data?

In [ ]:
pd.set_option('display.max_rows', df.shape[0])
data["kw_total_zone2"][data["kw_total_zone2"] == 0]

## Let's do some training

In [ ]:
from challenge_utils import build_training_data_from_df  

In [ ]:
target_time, target, predictors = build_training_data('filled_data.csv')
print(target.shape)
print(predictors.shape)

In [ ]:
data["kw_total_zone2"][7*24:15*24]

In [ ]:
def create_target_hourly_zone2(data):
    # Exclude the first column (datetime)
    input_data = data["kw_total_zone2"]

    # Calculate the number of days (excluding the first week)
    num_days = 730 - 7  # 730 days with the first 7 days skipped

    # Initialize the 3D array: 24 hour, 27 variable
    targets = np.zeros((num_days, 24))

    # Iterate through each day and extract the data
    for day in range(num_days):
        start_index = day * 24 + 7 * 24
        end_index = start_index + 24
        targets[day] = input_data.iloc[start_index:end_index]

    return targets # taking the kw_zone 2 to predict

hourly_consumption_zone2 = create_target_hourly_zone2(data)
print(hourly_consumption_zone2[:10])

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))

def scale_data(predictors, targets):
    # Reshape the array to (723 * 168, 27) to apply MinMaxScaler
    reshaped_predictors = predictors.reshape(-1, 27)

    # Fit and transform the data
    scaled_predictors = scaler.fit_transform(reshaped_predictors)

    # Reshape the scaled data back to the original shape
    scaled_predictors = scaled_predictors.reshape(predictors.shape)

    reshaped_target = targets.reshape(-1, 1)
    scaled_targets = scaler.fit_transform(reshaped_target)
    # target = scaler.fit_transform(reshaped_target)
    return scaled_predictors, scaled_targets

scaled_predictors, scaled_targets = scale_data(predictors, hourly_consumption_zone2)

In [ ]:
print(scaled_predictors.shape, scaled_targets.shape)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.layers import Lambda
import tensorflow.keras.backend as K

# TODO

- Try model with feature selection and feature engineering
    - Try fewer features with high correlation
    - Try constructing new features from 26 params
- Try extrapolate the kw_total_zone2 with periodic fitter because there are zeros in the data.
- Benchmark LSTM model with regression models like linear reg and decision tree with XGBoost